In [99]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

train = pd.read_csv('./data/train.csv', index_col='no')
test = pd.read_csv('./data/test.csv', index_col='no')


def strip_title(row):
    return row.strip()

# train 공백제거
# except pass 는 수치형나오면 오류나는거를 pass한것
for i in range(len(train.columns)):
    try:
        train.iloc[:,i] = train.iloc[:,i].apply(strip_title)
    except:
        pass
# test 공백제거
for i in range(len(test.columns)):
    try:
        test.iloc[:,i] = test.iloc[:,i].apply(strip_title)
    except:
        pass


# ?을 other로 변환
train['occupation'] = train['occupation'].replace('?','other')
train['workclass'] = train['workclass'].replace('?','other')
train['native-country'] = train['native-country'].replace('?','other')

# ?을 other로 변환
test['occupation'] = test['occupation'].replace('?','other')
test['workclass'] = test['workclass'].replace('?','other')
test['native-country'] = test['native-country'].replace('?','other')

#train.drop(['fnlwgt','capital-gain', 'capital-loss','education'],axis=1,inplace=True)
#test.drop(['fnlwgt','capital-gain', 'capital-loss','education'],axis=1,inplace=True)

#train.drop(['fnlwgt','education'],axis=1,inplace=True)
#test.drop(['fnlwgt','education'],axis=1,inplace=True)

train['native-country'] = train['native-country'].map({'United-States':'United-States',"other":'United-States'})
train['native-country'].fillna("other-country",inplace=True)
test['native-country'] = test['native-country'].map({'United-States':'United-States',"other":'United-States'})
test['native-country'].fillna("other-country",inplace=True)


In [100]:
# 인코딩할 컬럼명만 선택
categorical_features = ['workclass', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']
# categorical_features = ['workclass','occupation']
# 반복문으로 인코딩하기
for feature_name in categorical_features:
    one_hot = pd.get_dummies(train[feature_name], prefix = feature_name)
    train = pd.concat([train,one_hot], axis=1) # 기존 데이터 끝에 one_hot을 붙임.
    train.drop(feature_name, axis=1, inplace=True) # 기존 글자컬럼을 삭제
    
for feature_name in categorical_features:
    one_hot = pd.get_dummies(test[feature_name], prefix = feature_name)
    test = pd.concat([test,one_hot], axis=1) # 기존 데이터 끝에 one_hot을 붙임.
    test.drop(feature_name, axis=1, inplace=True) # 기존 글자컬럼을 삭제

In [61]:
train.shape

(29305, 54)

In [62]:
test.shape

(19537, 53)

In [101]:
train.drop("workclass_other", axis = 1, inplace = True)
test.drop("workclass_other", axis = 1, inplace = True)

In [102]:

train.sort_index(axis=1, inplace=True)
test.sort_index(axis=1, inplace=True)
X_train = train.drop(['income'], axis=1)
y_train = train.income
X_test=test
X_train.sort_index(axis=1, inplace=True)
X_test.sort_index(axis=1, inplace=True)


In [94]:

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, random_state=0)

In [95]:
X_train.shape

(21978, 52)

In [96]:
X_test['education'].value_counts()

1     2320
2     1666
3     1270
4      391
5      277
7      255
6      250
8      206
10     136
9      135
12     108
11     104
14      91
13      76
15      33
16       9
Name: education, dtype: int64

In [92]:
ed_dic={
    'HS-grad' : 1,
    'Some-college': 2,
    'Bachelors': 3,
    'Masters': 4,
    'Assoc-voc': 5,
    '11th': 6,
    'Assoc-acdm': 7,
    '10th': 8,
    '7th-8th':9,
    'Prof-school': 10,
    '9th':11,
    '12th':12,
    '5th-6th':13,
    'Doctorate':14,
    '1st-4th':15,
    'Preschool':16
}
X_train['education'] = X_train['education'].map(ed_dic)
X_test['education'] = X_test['education'].map(ed_dic)

In [83]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier

knn = KNeighborsClassifier()
lr = LogisticRegression(max_iter=100000)

# svm의 경우 soft 보팅이 실행 안됨 (확률예측 함수가 없음)
voting = VotingClassifier(estimators=[("KNN", knn), ("LR", lr)], voting = 'soft')

In [97]:
from sklearn.metrics import accuracy_score

voting.fit(X_train, y_train)
  

pred_vote = voting.predict(X_test)


print(accuracy_score(pred_vote, y_test))


0.795414221373004


In [24]:
pred_vote.shape

(7327,)

In [ ]:
import pandas as pd

In [27]:
y_test.shape

(7327,)

In [29]:
test

,age,capital-gain,capital-loss,education-num,hours-per-week,marital-status_Divorced,marital-status_Married-AF-spouse,marital-status_Married-civ-spouse,marital-status_Married-spouse-absent,marital-status_Never-married,...,sex_Female,sex_Male,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay
no,,,,,,,,,,,,,,,,,,,,,
29306,18,0,0,10,16,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
29307,29,0,0,9,40,0,0,1,0,0,...,1,0,0,0,0,1,0,0,0,0
29308,45,0,0,13,40,0,0,1,0,0,...,0,1,0,0,0,1,0,0,0,0
29309,42,0,0,13,40,0,0,1,0,0,...,0,1,0,0,0,0,0,1,0,0
29310,41,0,1902,16,40,0,0,1,0,0,...,1,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48838,45,0,0,9,60,0,0,1,0,0,...,0,1,0,0,0,0,0,1,0,0
48839,48,0,0,9,50,0,0,0,0,1,...,1,0,0,0,0,1,0,0,0,0
48840,63,4386,0,10,40,0,0,1,0,0,...,0,1,0,0,0,1,0,0,0,0


In [28]:
pre = voting.predict(test)

array([0, 0, 1, ..., 1, 0, 1], dtype=int64)

In [31]:
submission = pd.read_csv("./data/sample_submission.csv",index_col="no")
submission["income"] = pre
submission.to_csv('data1/bg1.csv')


In [98]:
!pip install catboost